# Convertible Bond Strategy - Factor 0003

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2022-12-31'
ed = '2026-02-13'

In [3]:
sql_factor = f"""
SELECT 
    date, 
    instrument,
    conversion_premium_rate AS factor,
FROM cn_cbond_basic_info JOIN cn_cbond_analyze_metric USING (instrument)
WHERE current_yield > 0.05
AND credit_rating = 'AAA'
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 20
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters={"date":[sd,ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2023-01-03,110053.SH,1.370111,1.0,0.05
1,2023-01-03,110061.SH,3.154435,2.0,0.05
2,2023-01-03,113050.SH,8.261431,3.0,0.05
3,2023-01-03,127032.SZ,8.900000,4.0,0.05
4,2023-01-03,110075.SH,10.464361,5.0,0.05
...,...,...,...,...,...
15135,2026-02-13,127110.SZ,40.947164,16.0,0.05
15136,2026-02-13,113067.SH,45.110371,17.0,0.05
15137,2026-02-13,127049.SZ,49.290612,18.0,0.05
15138,2026-02-13,127056.SZ,85.288773,19.0,0.05


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 21:12:29] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 21:12:29] [info     ] 2023-01-03, 2026-02-13, , , instruments=43
[2026-02-18 21:12:29] [info     ] bigtrader module V2.2.0
[2026-02-18 21:12:29] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '110808.SH' at '2023-03-08 15:00:00'
order_target_percent: price is null for '113011.SH' at '2023-03-14 15:00:00'
order_target_percent: price is null for '113011.SH' at '2023-03-15 15:00:00'
order_target_percent: price is null for '113011.SH' at '2023-03-16 15:00:00'
order_target_percent: price is null for '113011.SH' at '2023-03-17 15:00:00'
order_target_percent: price is null for '110808.SH' at '2023-03-24 15:00:00'
order_target_percent: price is null for '110808.SH' at '2023-03-27 15:00:00'
order_target_percent: price is null for '110808.SH' at '2023-03-28 15:00:00'
order_target_percent: price is null for '110808.SH' at '2023-03-30 15:00:00'
order_target_percent: price is n

[2026-02-18 21:12:35] [info     ] bigtrader.v34 运行完成 [5.844s].
